In [1]:
import numpy as np
import matplotlib.pyplot as plt
import data
from data import DURATION, WIDTH, HEIGHT


In [4]:
input_field_names = ["psl", "vas", "sfcWind"]
d = len(input_field_names)
input_fields = [data.load_field(name)[1] for name in input_field_names]
all_inputs = np.concatenate([f.values for f in input_fields], axis=3)
all_inputs = all_inputs.reshape(DURATION, HEIGHT, WIDTH, d)

_, wsgsmax_field = data.load_field("wsgsmax")
all_wsgsmax = wsgsmax_field.values.reshape(DURATION, HEIGHT, WIDTH)

time = np.arange(500, 500 + 720, 1)
x = 40
y = 50

inputs = all_inputs[time, y, x, :]
wsgsmax = all_wsgsmax[time, y, x]

In [ ]:
fig, axes = plt.subplots(len(input_field_names))
for i, (name, ax) in enumerate(zip(input_field_names, axes)):
    ax.scatter(inputs[:, i], wsgsmax)
    ax.set_ylabel(name)
    ax.set_xlabel("wsgsmax")
plt.show()